In [1]:
import os, os.path
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize
import pandas as pd
from collections import Counter
import numpy as np

#  Create documents

In [4]:
# creating a .tsv document for each review
with open('Airbnb_Texas_Rentals.tsv', 'r', encoding = "utf8") as all_rev:
    csv_reader = csv.reader(all_rev)
    ind = -1
    for row in all_rev:
        ind += 1
        if ind == 0:
            continue
        # skipping the empty lines
        elif row == '\n':
            ind -= 1
            # we manipulate 'ind' in order to have files named doc_i where i = 1,2,3,..
        else:
            # we store the documents inside a new folder
            new_title = r'fileAIR\doc_' + str(ind-1) + '.tsv'
            csv_writer = csv.writer(open(new_title, 'w', encoding = "utf8"), delimiter ='\t')
            # we skip the first column which is 
            row1 = row.split("\t")[1:]
            csv_writer.writerow(row1)

KeyboardInterrupt: 

# Search Engine 1

In [2]:
# function to preprocess the files
def preprocess(something):
    something = something.lower()
    # removing '\n'
    something = something.replace('\\n', ' ')
    # removing punctuation
    tokenizer = regexp_tokenize(something, "[\w'\$]+")
    # filter the non stopwords
    filtered = [w for w in tokenizer if not w in stopwords.words('english')]
    ps = PorterStemmer()
    # removing the stem
    filtered = [ps.stem(word) for word in filtered]
    return filtered

In [3]:
len_file = len([x for x in os.scandir(r'C:\Users\alice\Desktop\ADM_HW3\fileAIR')])
print (len_file)

1001


In [4]:
vocabulary_set = set()
docs_list = []

In [5]:
for i in range(len_file):
    with open(r'C:\Users\alice\Desktop\ADM_HW3\fileAIR\doc_' + str(i) + '.tsv', 'r', encoding = 'utf8') as csvfile:
        file1 = csv.reader(csvfile, delimiter = '\t')
        columns = [i for i in file1]
        # we want to focus only on description and title
        description = columns[0][4]
        description = preprocess(description)
        title = columns[0][7]
        title = preprocess(title)
        tit_desc = title + description
        # creating the vocabulary
        vocabulary_set.update(tit_desc)
        docs_list.append(set(tit_desc))

In [6]:
vocabulary = {k:v for v, k in enumerate(vocabulary_set)}

In [7]:
# save vocabulary to the file
voc_file = open(r'C:\Users\alice\Desktop\ADM_HW3\vocabulary.txt', 'w', encoding = "utf8")
for term in vocabulary:
    voc_file.write('{0}\t{1}\n'.format(term, vocabulary[term]))
    # we have to close the file

In [9]:
# using defaultdict because it gives no key error
from collections import defaultdict 
# we need to sort this value
inv_indx = defaultdict(set)
for idx, text in enumerate(docs_list):
    for word in text:
        id_word = vocabulary[word]
        inv_indx[id_word].add(idx)

In [10]:
# to have a list as the value inside the dict
for key in inv_indx:
    value = inv_indx[key]
    inv_indx[key] = list(value)

In [11]:
# save inv_indx to the .txt file
inv_file = open(r'C:\Users\alice\Desktop\ADM_HW3\inverted_indx.txt', 'w', encoding = "utf8")
for id_term in inv_indx:
    docks = inv_indx[id_term]
    d = '\t'.join(map(str, docks))
    inv_file.write('{0}\t{1}\n'.format(id_term, d))

In [12]:
query = str(input())

garden house


In [13]:
def SearchEngine(query, vocab, inv_indx):
    li = preprocess(query)
    pos_list = []
    for item in li:
        if item not in vocabulary:
            print('No documents found')
            break
        pos_id = vocabulary[item]
        pos_list.append(pos_id)
    result_list = []
    for term_id in pos_list:
        result_list.append(inv_indx[term_id])
    return result_list
# do we have to give an error when no document is found?

In [14]:
def findTheSomething(res_list, pre_query):
    test = {k:1 for k in res_list[0]}
    for sublist_ind in range(1, len(res_list)):
        for k in res_list[sublist_ind]:
            try:
                test[k] += 1
            except:
                test[k] = 1
    return [doc_id for doc_id in test if test[doc_id] == len(pre_query)]

In [15]:
pos = SearchEngine(query, vocabulary, inv_indx)

In [16]:
doc = findTheSomething(pos, preprocess(query))
#doc

In [17]:
#doc = [130, 137, 103]
g = []
for i in doc:
    g.append(pd.read_csv(r'fileAIR\doc_' + str(i) + '.tsv', sep = '\t'))
g1 = [[j for j in i] for i in g]
cols = ['1', '2', 'City', '4','Description','6','7','Title','Url']
df1 = pd.DataFrame(g1, columns = cols)[['Title','Description','City','Url']]
df1

,Title,Description,City,Url
0,Unique Location! Alamo Heights - Designer Insp...,"Stylish, fully remodeled home in upscale NW – ...",San Antonio,https://www.airbnb.com/rooms/17481455?location...
1,Comfy Private Room in Fort Worth,Our house is the perfect place to stay in Fort...,Fort Worth,https://www.airbnb.com/rooms/2502098?location=...
2,a DRAMATIC 3 level LOFT APARTMENT,"The 3 LEVEL LOFT APARTMENT, a DRAMATIC, UNIQUE...",Houston,https://www.airbnb.com/rooms/2714950?location=...
3,Mid-century spaciousness,Large and airy private wing of house. Experien...,Alpine,https://www.airbnb.com/rooms/10726330?location...
4,Beautiful king bedroom in NW Austin,"My house is close to Lakeline Mall, highways a...",Austin,https://www.airbnb.com/rooms/15162593?location...
5,Oak Cliff Tiny House,Warm open space guesthouse cultured as a medit...,Dallas,https://www.airbnb.com/rooms/8799838?location=...
6,1 Bedroom in quiet neighborhood,Private bedroom with queen size bed and privat...,Saginaw,https://www.airbnb.com/rooms/8305468?location=...
7,Private Rm/Bath @ the Gold House!,This unique private guest room at the Gold Hou...,San Antonio,https://www.airbnb.com/rooms/1284796?location=...
8,Oak Cliff Tiny House,Warm open space guesthouse cultured as a medit...,Dallas,https://www.airbnb.com/rooms/8799838?location=...


# Search Engine 2

In [18]:
global final_dict
final_dict = {}

def computeTFIDF(freq_dict, doc_id):
    numWords = len(freq_dict)
    for word in freq_dict.keys():
        word_id = vocabulary[word]
        try:
            final_dict[word_id].append((doc_id, round(float(freq_dict[word])/numWords, 3)))
        except:
            final_dict[word_id] = [(doc_id, round(float(freq_dict[word])/numWords, 3))]
    return 

In [52]:
def occurenceNum(index):
    with open(r'C:\Users\alice\Desktop\ADM_HW3\fileAIR\doc_' + str(index) + '.tsv', 'r', encoding = "utf8") as doc:
        file1 = csv.reader(doc, delimiter = '\t')
        columns = [i for i in file1]
        description = columns[0][4]
        description = preprocess(description)
        title = columns[0][7]
        title = preprocess(title)
        tit_desc = title + description
        freq_dict = {}
        for w in tit_desc:
            try:
                freq_dict[w] += 1
            except:
                freq_dict[w] = 1
        return computeTFIDF(freq_dict, index)

In [54]:
final_inv_dict = []
for file in range(len_file):
    test = occurenceNum(file)
    final_inv_dict.append(test)

In [55]:
final_dict

{3033: [(0, 0.024),
  (1, 0.024),
  (2, 0.024),
  (3, 0.024),
  (4, 0.024),
  (5, 0.024),
  (6, 0.024),
  (7, 0.024),
  (8, 0.024),
  (9, 0.024),
  (10, 0.024),
  (11, 0.024),
  (12, 0.024),
  (13, 0.024),
  (14, 0.024),
  (15, 0.024),
  (16, 0.024),
  (17, 0.024),
  (18, 0.024),
  (19, 0.024),
  (20, 0.024),
  (21, 0.024),
  (22, 0.024),
  (23, 0.024),
  (24, 0.024),
  (25, 0.024),
  (26, 0.024),
  (27, 0.024),
  (28, 0.024),
  (29, 0.024),
  (30, 0.024),
  (31, 0.024),
  (32, 0.024),
  (33, 0.024),
  (34, 0.024),
  (35, 0.024),
  (36, 0.024),
  (37, 0.024),
  (38, 0.024),
  (39, 0.024),
  (40, 0.024),
  (41, 0.024),
  (42, 0.024),
  (43, 0.024),
  (44, 0.024),
  (45, 0.024),
  (46, 0.024),
  (47, 0.024),
  (48, 0.024),
  (49, 0.024),
  (50, 0.024),
  (51, 0.024),
  (52, 0.024),
  (53, 0.024),
  (54, 0.024),
  (55, 0.024),
  (56, 0.024),
  (57, 0.024),
  (58, 0.024),
  (59, 0.024),
  (60, 0.024),
  (61, 0.024),
  (62, 0.024),
  (63, 0.024),
  (64, 0.024),
  (65, 0.024),
  (66, 0.024),

In [56]:
# save inv_indx to the .txt file
inv_file_score = open(r'C:\Users\alice\Desktop\ADM_HW3\inverted_indx_tfid.txt', 'w', encoding = "utf8")
for id_term in final_dict:
    docks = final_dict[id_term]
    d = '\t'.join(map(str, docks))
    inv_file_score.write('{0}\t{1}\n'.format(id_term, d))

In [70]:
# computing cosine similarity
def cosine_sim(x, y):
    dot_product = np.dot(x, y)
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return dot_product / (norm_x * norm_y)

In [71]:
def query_tfid(prep_query_list):
    len_list = len(prep_query_list) 
    test = [prep_query_list.count(word)/len_list for word in prep_query_list]
    return test

In [85]:
def get_tfid(arg_list, doc_id):
    for tuple_ in arg_list:
        #print(tuple_[0], doc_id)
        if tuple_[0] == doc_id:
            return tuple_[1]

In [87]:
def SearchEngine_new(query, vocabulary,inv_indx, inv_indx_score):
    query_list = preprocess(query)
    pos_list = []
    for item in query_list:
        if item not in vocabulary:
            print('No documents found')
            break
        pos_id = vocabulary[item]
        pos_list.append(pos_id)
    result_list = []
    for term_id in pos_list:
        result_list.append(inv_indx[term_id])
    selected_docs = findTheSomething(result_list, query_list)
    # calculating cosine similarity -------
    cos_sim_list = []
    for doc_id in selected_docs:
        
        # creating the tfid vector for a document ------
        tfid_vector = []
        for word in pos_list: #for every word in the query
            g = get_tfid(inv_indx_score[word], doc_id)
            tfid_vector.append(g)
        
        #pos_list - list of id of each work in query
                          
        #tfid_vector = [(test, doc_id) for word in pos_list]
        cos_sim_list.append(cosine_sim(query_tfid(pos_list), tfid_vector))
    return result_list, cos_sim_list

In [61]:
query = str(input())

private bedroom


In [88]:
SearchEngine_new(query, vocabulary, inv_indx, final_dict)

[0.8574929257125441,
 0.9486832980505138,
 1.0,
 1.0,
 0.9797619135918656,
 1.0,
 0.8919700836007997,
 0.9902407678035506,
 1.0,
 0.9520285612852641,
 0.9999999999999999,
 0.9486832980505138,
 0.8320502943378436,
 0.9472159695726096,
 0.9999999999999999,
 0.9472810325086992,
 1.0,
 0.9486832980505137,
 1.0,
 1.0,
 1.0,
 0.945537585866072,
 1.0,
 1.0,
 0.8337123014813119,
 1.0,
 0.8924127761526154,
 1.0,
 0.9805806756909202,
 0.9999999999999999,
 0.9999999999999999,
 0.9511248018899605,
 1.0,
 1.0,
 0.9999999999999999,
 1.0,
 1.0,
 0.9467132382195149,
 0.9459459459459459,
 0.9999999999999999,
 0.9472159695726096,
 0.9486832980505138,
 1.0,
 1.0,
 0.9501926425807355,
 0.9486832980505137,
 1.0,
 0.9486832980505137,
 0.9486832980505138,
 0.9999999999999998,
 1.0,
 0.9470234336955625,
 0.8956763762919447,
 0.8932564867578889,
 0.9486832980505138,
 0.9462604796066584,
 1.0,
 0.9514445185544115,
 1.0,
 0.9472159695726096,
 0.9504448598378739,
 1.0,
 0.9805806756909201,
 1.0,
 0.94868329805051